In [ ]:
#1.
import pandas as pd

#df_terms = pd.read_excel('wordcount.xlsx')
df_terms = pd.read_excel('coding2.1 übersicht level1.xlsx')
#df_terms = pd.read_excel('test_1.xlsx')

#df_terms=df_terms.loc[df_terms['Level1'] == 'Attitude & Behaviour']
#df_terms=df_terms.loc[df_terms['brand'] == 'BMW']
df_terms = df_terms.reset_index()
df_terms=df_terms.drop(['index','brand'], axis = 1)

In [ ]:
df_terms

In [ ]:
#2.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
example_sent = """This is a sample sentence,
                  showing off the stop words filtration."""
 
stop_words = set(stopwords.words('english'))
 
f = lambda x: ' '.join([item for item in x.split() if item not in stop_words])
df_terms["clean_sentence"] = df_terms["clean_sentence"].apply(f)

stop_words = ["\n","\""]
df_terms["clean_sentence"] = df_terms["clean_sentence"].apply(f)

In [ ]:
#3 library
def num_there(s):
    return any(i.isdigit() for i in s)

def common_start(sa, sb):
    """ returns the longest common substring from the beginning of sa and sb """
    def _iter():
        for a, b in zip(sa, sb):
            if a == b:
                yield a
            else:
                return

    return ''.join(_iter())

def my_common_kernel(sa, sb):

    mySubstrings = []
    myKernelStrings = []
    minSize = int(len(sa)*0.8)
    #myString = input("Enter a string: ")
    
    if len(sa)/len(sb)<0.7: return 

    actualLength = 0
    kernelString = ""

    for i in range(len(sa)-minSize):
        mySubstrings.append(sa[0:len(sa)-i])
                
    for j in range(len(mySubstrings)):
        for i in range(len(sb)-minSize+1):
            kernelString = common_start(sb[0:len(mySubstrings[j])], mySubstrings[j])
            if len(kernelString)>minSize: myKernelStrings.append(kernelString)
                
    if len(myKernelStrings)>0: 
        return max(myKernelStrings, key=len)
    

In [ ]:
subcategories = df_terms['Level2'].unique()
subcategories

In [ ]:
# 4. ermittle top terms pro Level2 categorie
for i in range(len(subcategories)):
    print(subcategories[i])    
    df_terms2=df_terms.loc[df_terms['Level2'] == subcategories[i]]
    df_terms2 = df_terms2.reset_index(drop=False)
    
    alle_terms_liste = []
    
    # separiere alle Wörter aus allen Texten in eine Liste
    for j in range(len(df_terms2)):
        topic_term_liste = df_terms2.loc[j]['clean_sentence'].split() 
        alle_terms_liste = alle_terms_liste + topic_term_liste

    #myString = input("Enter a string: ")
    new_terms = []
    # entferne alle Zahlen und wörter unter 3 Buchstaben
    for k in range(len(alle_terms_liste)):
        if len(alle_terms_liste[k]) > 2 and not num_there(alle_terms_liste[k]): 
            new_terms.append(alle_terms_liste[k])
    
    lemmatized_List = []
    #myString = input("Enter a string: ")  
    myTerms = new_terms
    
    while (len(myTerms)>0):

        lemmatized_List_tmp = []
        myTerms_tmp = []

        lemmatized_List_tmp.append([myTerms[0], myTerms[0]])

        for word in myTerms[1:]:

            myKernel = my_common_kernel(myTerms[0], word)
            if myKernel: 
                lemmatized_List_tmp.append([word, myKernel])
            else:
                myTerms_tmp.append(word)

        columns = ['word','lemma']

        df = pd.DataFrame(data=lemmatized_List_tmp,columns=columns)

        kernel=min(df["lemma"], key=len)
        df['real_lemma'] = kernel
        df=df.drop(['lemma'], axis = 1)
        lemmatized_List_tmp=df.values.tolist()

        lemmatized_List.extend(lemmatized_List_tmp)

        myTerms = myTerms_tmp
        #print(len(myTerms))

    columns = ['word','lemma']
    dftmp = pd.DataFrame(data=lemmatized_List,columns=columns)

    dff = pd.DataFrame(data=dftmp['lemma'].value_counts())
    dff['top_term'] = dff.index
    
    dff.rename(columns={'lemma': 'count'}, inplace=True)
    dff=dff.head(40)
    dff = dff[['top_term', 'count']]
    dff[subcategories[i]] = dff['top_term'] + " " + dff['count'].astype(str)
    dff = dff.drop(['top_term', 'count'], axis=1)
    dff = dff.reset_index(drop=True)
    
    if i==0:
        columns = [subcategories[i]]
        df_full = pd.DataFrame(data=dff[subcategories[i]].tolist(),columns=[subcategories[i]])
    else:
        df_full[subcategories[i]]=dff[subcategories[i]]
    
    df_full = df_full.append(dff)
    

In [ ]:
#pd.set_option('display.max_rows', None)
df_full

In [ ]:
df_full.to_excel('Attitude&behaviour_topterms_per_subcategory.xlsx')   